In [1]:
#coding:utf-8
import tensorflow as tf
import sys,time
import numpy as np
import pickle as cPickle
import os
import random
import Config
import Model
import codecs

In [2]:
config_tf = tf.ConfigProto()
config_tf.gpu_options.allow_growth = True
config_tf.inter_op_parallelism_threads = 1
config_tf.intra_op_parallelism_threads = 1

In [3]:
config = Config.Config()

In [4]:
char_to_idx, idx_to_char = cPickle.load(open(config.model_path+'.voc', 'rb'))

config.vocab_size = len(char_to_idx)

In [5]:
is_sample = config.is_sample
is_beams = config.is_beams
beam_size = config.beam_size
len_of_generation = config.len_of_generation
print(len_of_generation)
start_sentence = config.start_sentence

10


In [6]:
is_sample = True
is_beams = True
beam_size = 4
len_of_generation = 200

#start_sentence = "阿貓阿狗"
#start_sentence = "生似蜉蝣"
#start_sentence = "忘掉种过的花"
start_sentence = "如果"


In [7]:
if (len(sys.argv) == 2):
    if (sys.version_info > (3, 0)):
        start_sentence = sys.argv[1]
    else:
        start_sentence = sys.argv[1].decode("utf-8")

In [8]:
def run_epoch(session, m, data, eval_op, state=None):
    """Runs the model on the given data."""
    x = data.reshape((1,1))
    output,prob, _state, _ = session.run([m._outputs, m._prob, m.final_state, eval_op],
                         {m.input_data: x,
                          m.initial_state: state})
    
    #print("output shape:",len(output),len(output[0]))
    #print("outputs:",type(output),output)

    return prob, _state

In [9]:
def main(_):
    with tf.Graph().as_default(), tf.Session(config=config_tf) as session:
        config.batch_size = 1
        config.num_steps = 1

        initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
        with tf.variable_scope("model", reuse=None, initializer=initializer):
            mtest = Model.Model(is_training=False, config=config)

        #tf.global_variables_initializer().run()

        model_saver = tf.train.Saver()
        print("model loading ...")
        model_saver.restore(session, config.model_path+'-%d'%config.save_time)
        print ("Done!")
        
        print("is_beams:",is_beams)

        if not is_beams:
            # sentence state
            char_list = list(start_sentence);
            start_idx = char_to_idx[char_list[0]]
            _state = mtest.initial_state.eval()
            test_data = np.int32([start_idx])
            prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), _state)
            print("prob:",type(prob),len(prob[0]),prob)
            print("state:",type(_state),len(_state[0]),_state)
            gen_res = [char_list[0]]
            for i in range(1, len(char_list)):
                char = char_list[i]
                try:
                    char_index = char_to_idx[char]
                except KeyError:
                    char_index = np.argmax(prob.reshape(-1))
                prob, _state = run_epoch(session, mtest, np.int32([char_index]), tf.no_op(), _state)
                gen_res.append(char)
            
            print("This is gen_res:",gen_res)


            # gen text
            if is_sample:
                gen = np.random.choice(config.vocab_size, 1, p=prob.reshape(-1))
                gen = gen[0]
            else:
                gen = np.argmax(prob.reshape(-1))
            test_data = np.int32(gen)
            gen_res.append(idx_to_char[gen])
            for i in range(len_of_generation-1):
                prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), _state)
                if is_sample:
                    gen = np.random.choice(config.vocab_size, 1, p=prob.reshape(-1))
                    gen = gen[0]
                else:
                    gen = np.argmax(prob.reshape(-1))
                test_data = np.int32(gen)
                gen_res.append(idx_to_char[gen])
            print("Generated Result: {0}".format(gen_res))
            print("".join(gen_res))
        else:
            # sentence state
            char_list = list(start_sentence);
            start_idx = char_to_idx[char_list[0]]
            _state = mtest.initial_state.eval()
            beams = [(0.0, [idx_to_char[start_idx]], idx_to_char[start_idx])]
            test_data = np.int32([start_idx])
            prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), _state)
            y1 = np.log(1e-20 + prob.reshape(-1))
            beams = [(beams[0][0], beams[0][1], beams[0][2], _state)]
            for i in range(1, len(char_list)):
                char = char_list[i]
                try:
                    char_index = char_to_idx[char]
                except KeyError:
                    top_indices = np.argsort(-y1)
                    char_index = top_indices[0]
                prob, _state = run_epoch(session, mtest, np.int32([char_index]), tf.no_op(), beams[0][3])
                y1 = np.log(1e-20 + prob.reshape(-1))
                beams = [(beams[0][0], beams[0][1] + [char], char_index, _state)]
            # gen text
            if is_sample:
                top_indices = np.random.choice(config.vocab_size, beam_size, replace=False, p=prob.reshape(-1))
            else:
                top_indices = np.argsort(-y1)
            b = beams[0]
            beam_candidates = []
            for i in range(beam_size):
                wordix = top_indices[i]
                beam_candidates.append((b[0] + y1[wordix], b[1] + [idx_to_char[wordix]], wordix, _state))
            beam_candidates.sort(key = lambda x:x[0], reverse = True) # decreasing order
            beams = beam_candidates[:beam_size] # truncate to get new beams
            for xy in range(len_of_generation-1):
                beam_candidates = []
                for b in beams:
                    test_data = np.int32(b[2])
                    prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), b[3])
                    y1 = np.log(1e-20 + prob.reshape(-1))
                    if is_sample:
                        top_indices = np.random.choice(config.vocab_size, beam_size, replace=False, p=prob.reshape(-1))
                    else:
                        top_indices = np.argsort(-y1)
                    for i in range(beam_size):
                        wordix = top_indices[i]
                        beam_candidates.append((b[0] + y1[wordix], b[1] + [idx_to_char[wordix]], wordix, _state))
                beam_candidates.sort(key = lambda x:x[0], reverse = True) # decreasing order
                beams = beam_candidates[:beam_size] # truncate to get new beams

            if (sys.version_info > (3, 0)):
                print("Generated Result: {0}".format(beams[0][1]))
                print("".join(beams[0][1]))
                #with open("writing/"+start_sentence+"_2.txt","w") as wf:
                #    wf.write("".join(beams[0][1]))
            else:
                print('Generated Result:')
                print(''.join(beams[0][1]))


if __name__ == "__main__":
    tf.app.run()

num_steps: 1
This is lstm_state size: 2000
This is lstm_output size: 1000
This is state size: 6000
This is output size: 1000
model loading ...
INFO:tensorflow:Restoring parameters from ./model/Model-40
Done!
is_beams: True


InternalError: Blas GEMM launch failed : a.shape=(1, 2000), b.shape=(2000, 4000), m=1, n=4000, k=2000
	 [[Node: model/RNN/RNN/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model/RNN/RNN/multi_rnn_cell/cell_0/basic_lstm_cell/concat, model/RNN/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/read)]]

Caused by op 'model/RNN/RNN/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ginko/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ginko/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ginko/.local/lib/python3.5/site-packages

In [ ]:
char_list = list(start_sentence);
print(char_list)
start_idx = char_to_idx[char_list[0]]
print(start_idx)
test_data = np.int32([start_idx])
print(test_data)
test_data.reshape((1,1))
print(test_data)

prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), _state)
print(prob)

In [ ]:
def main(_):
    with tf.Graph().as_default(), tf.Session(config=config_tf) as session:
        config.batch_size = 1
        config.num_steps = 1

        initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
        with tf.variable_scope("model", reuse=None, initializer=initializer):
            mtest = Model.Model(is_training=False, config=config)

        #tf.global_variables_initializer().run()

        model_saver = tf.train.Saver()
        print("model loading ...")
        model_saver.restore(session, config.model_path+'-%d'%config.save_time)
        print ("Done!")

        if not is_beams:
            # sentence state
            char_list = list(start_sentence);
            start_idx = char_to_idx[char_list[0]]
            _state = mtest.initial_state.eval()
            test_data = np.int32([start_idx])
            prob, _state = run_epoch(session, mtest, test_data, tf.no_op(), _state)
            gen_res = [char_list[0]]
            for i in range(1, len(char_list)):
                char = char_list[i]
                try:
                    char_index = char_to_idx[char]
                except KeyError:
                    char_index = np.argmax(prob.reshape(-1))
                prob, _state = run_epoch(session, mtest, np.int32([char_index]), tf.no_op(), _state)
                gen_res.append(char)
            #print(gen_res)
                



if __name__ == "__main__":
    tf.app.run()